In [1]:
import csv
import cv2
from pathlib import Path
from deepface import DeepFace
from authentication import login, signup

class EmotionDetector:
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def detect_faces(self, image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return self.face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    def analyze_emotion(self, face_roi):
        result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
        if isinstance(result, list):
            result = result[0]
        return result['dominant_emotion']

    def draw_rectangle(self, image, x, y, w, h, emotion):
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(image, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

class ImageEmotionDetector(EmotionDetector):
    def __init__(self):
        super().__init__()

    def detect_emotions_in_image(self, image_path):
        try:
            if isinstance(image_path, (str, Path)):
                image_path = str(image_path)
            else:
                raise ValueError("Invalid image path type. Must be a string or Path object.")
                
            image = cv2.imread(image_path)
            if image is None:
                print("Error: Image not found or unable to read.")
                return

            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            faces = self.detect_faces(rgb_image)

            results = []
            for (x, y, w, h) in faces:
                face_roi = rgb_image[y:y + h, x:x + w]
                emotion = self.analyze_emotion(face_roi)
                self.draw_rectangle(image, x, y, w, h, emotion)
                results.append({'Image Path': image_path, 'Emotion': emotion})

            cv2.imshow('Emotion Recognition - Image', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            with open('emotion_results.csv', 'a', newline='') as csvfile:
                fieldnames = ['Image Path', 'Emotion']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                if csvfile.tell() == 0:
                    writer.writeheader()

                writer.writerows(results)

            print("Detected emotions written to 'emotion_results.csv'")

        except Exception as e:
            print(f"An error occurred: {e}")

class VideoEmotionDetector(EmotionDetector):
    def __init__(self):
        super().__init__()

    def detect_emotions_in_video(self):
        cap = cv2.VideoCapture(0)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rgb_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
            faces = self.detect_faces(rgb_frame)

            for (x, y, w, h) in faces:
                face_roi = rgb_frame[y:y + h, x:x + w]
                emotion = self.analyze_emotion(face_roi)
                self.draw_rectangle(frame, x, y, w, h, emotion)

            cv2.imshow('Real-time Emotion Recognition', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

def main():
    while True:
        choice = input("Enter '1' to login or '2' to signup: ")

        if choice == '1':
            username = input("Enter your username: ")
            password = input("Enter your password: ")
            if not login(username, password):
                print("Invalid username or password. Exiting...")
                continue
            break
        elif choice == '2':
            signup()
            print("Signup successful. Please login.")
        else:
            print("Invalid choice. Please try again.")

    while True:
        choice = input("Enter '1' to upload an image or '2' to detect emotions in a video: ")

        if choice == '1':
            image_path = input("Enter the path of the image: ")
            detector = ImageEmotionDetector()
            detector.detect_emotions_in_image(image_path)
            break
        elif choice == '2':
            detector = VideoEmotionDetector()
            detector.detect_emotions_in_video()
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


Login successful!
Detected emotions written to 'emotion_results.csv'
